**Introduction**
  We propose to achieve movie genre classification based only on movie poster images. A deep neural network is constructed to jointly describe visual appearance and object information, and classify a given movie poster image into genres. Because a movie may belong to multiple genres, this is a multi-label image classification problem. To facilitate related studies, we collect a large-scale movie poster dataset, associated with various metadata. Based on this dataset, we fine-tune a pre-trained convolutional neural network to extract visual representation and adopt a state-of-the-art framework to detect objects in posters. Two types of information are then integrated by the proposed neural network. In the evaluation, we show that the proposed method yields encouraging performance, which is much better than previous works. "

In [1]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 38kB/s 
     |████████████████████████████████| 3.8MB 43.7MB/s 
     |████████████████████████████████| 450kB 50.7MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


# New Section

# New Section

In [0]:
import pandas as pd
import json
import glob  
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D


In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [0]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [5]:
print(tf.__version__)

2.0.0


In [6]:
!git clone https://github.com/sweetcoonoor/Multiclassification_Genre.git

Cloning into 'Multiclassification_Genre'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 829 (delta 1), reused 0 (delta 0), pack-reused 822
Receiving objects: 100% (829/829), 24.18 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [0]:
!git clone https://github.com/laxmimerit/Movies-Poster_Dataset.git

Cloning into 'Movies-Poster_Dataset'...
remote: Enumerating objects: 7873, done.
remote: Counting objects: 100% (7873/7873), done.
remote: Compressing objects: 100% (7873/7873), done.
remote: Total 7873 (delta 1), reused 7872 (delta 0), pack-reused 0
Receiving objects: 100% (7873/7873), 246.59 MiB | 38.64 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (7871/7871), done.


In [0]:
ff=open("/content/Multiclassification_Genre/1980.txt","r")
Movies=ff.read()
comma=Movies.replace("}","},")
newc=comma[:-2]
sqaureAdd="["+newc+"]"
aaa=sqaureAdd.replace("\n","")
jj=aaa.replace("ObjectId(","").replace("\")","\"")
fin=json.loads(jj)
df = pd.DataFrame.from_dict(fin, orient='columns')

In [0]:
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
dfFin
dfFin2=dfFin

In [14]:
all_files = glob.glob("/content/Multiclassification_Genre/*.txt")

print((all_files))
count=0
for files in all_files:
  try:
    ff=open(files,"r",encoding='utf-16')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
  except:
    ff=open(files,"r",encoding='utf-8')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
    

['/content/Multiclassification_Genre/1994.txt', '/content/Multiclassification_Genre/1981.txt', '/content/Multiclassification_Genre/1999.txt', '/content/Multiclassification_Genre/2009.txt', '/content/Multiclassification_Genre/2010.txt', '/content/Multiclassification_Genre/1998.txt', '/content/Multiclassification_Genre/1996.txt', '/content/Multiclassification_Genre/1986.txt', '/content/Multiclassification_Genre/2011.txt', '/content/Multiclassification_Genre/1984.txt', '/content/Multiclassification_Genre/1982.txt', '/content/Multiclassification_Genre/2015.txt', '/content/Multiclassification_Genre/1980.txt', '/content/Multiclassification_Genre/1989.txt', '/content/Multiclassification_Genre/1992.txt', '/content/Multiclassification_Genre/2006.txt', '/content/Multiclassification_Genre/1991.txt', '/content/Multiclassification_Genre/1993.txt', '/content/Multiclassification_Genre/2008.txt', '/content/Multiclassification_Genre/1997.txt', '/content/Multiclassification_Genre/2013.txt', '/content/Mu

In [15]:
Movie_data=pd.concat([dfFin,dfFin2])
#Movie_data.head()
Movie_data.shape


(6984, 20)

Data Exploration and Pre-processing

In [16]:
Movie_Final=Movie_data[['imdbID','Title','Genre',]]
Movie_Final.shape

(6984, 3)

In [0]:
Movie_Final.to_csv('Movie_genre.csv',encoding="ISO-8859-1")




In [0]:
#Onehot encoding
df = pd.read_csv("Movie_genre.csv")

df.dropna(subset=['Genre'],inplace=True)
df.drop_duplicates(subset=['imdbID'], keep='first', inplace=True)
df.set_index('imdbID',inplace=True)
target = df['Genre'].apply(lambda x: [w.strip() for w in x.split(',')])
genres = pd.get_dummies(target.apply(pd.Series).stack()).sum(level=0)


In [19]:
df

,Unnamed: 0,Title,Genre
imdbID,,,
tt0080684,0,Star Wars: Episode V - The Empire Strikes Back,"Action, Adventure, Fantasy"
tt0081562,1,Stir Crazy,"Comedy, Crime"
tt0080339,2,Airplane!,Comedy
tt0080377,3,Any Which Way You Can,"Action, Comedy"
tt0081375,4,Private Benjamin,"Comedy, War"
tt0080549,5,Coal Miner's Daughter,"Biography, Drama, Music"
tt0081529,6,Smokey and the Bandit II,"Action, Comedy"
tt0080453,7,The Blue Lagoon,"Adventure, Drama, Romance"
tt0080455,8,The Blues Brothers,"Action, Comedy, Crime"


In [35]:

target.shape
Movie_Final

,imdbID,Title,Genre
0,tt0080684,Star Wars: Episode V - The Empire Strikes Back,"Action, Adventure, Fantasy"
1,tt0081562,Stir Crazy,"Comedy, Crime"
2,tt0080339,Airplane!,Comedy
3,tt0080377,Any Which Way You Can,"Action, Comedy"
4,tt0081375,Private Benjamin,"Comedy, War"
5,tt0080549,Coal Miner's Daughter,"Biography, Drama, Music"
6,tt0081529,Smokey and the Bandit II,"Action, Comedy"
7,tt0080453,The Blue Lagoon,"Adventure, Drama, Romance"
8,tt0080455,The Blues Brothers,"Action, Comedy, Crime"
9,tt0081283,Ordinary People,Drama


In [43]:
Movie_genre=Movie_Final
Movie_genre['Genre']=[elem[0:5] for elem in Movie_genre['Genre']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
Movie_genre['Genre']

In [0]:
Moviegenrestats=Movie_genre.groupby('Genre').size().reset_index(name='counts')

In [45]:
Moviegenrestats

,Genre,counts
0,A,1368
1,B,432
2,C,2952
3,D,1440
4,F,216
5,H,288
6,M,144
7,R,72
8,W,72


In [0]:
Moviegenrestats=Moviegenrestats.sort_values('counts',ascending=False)

In [48]:
Moviegenrestats

,Genre,counts
2,C,2952
3,D,1440
0,A,1368
1,B,432
5,H,288
4,F,216
6,M,144
7,R,72
8,W,72
